# Homework set 7 Comp Econ
## Asset pricing applications of the Banach contraction mapping theorem (in Julia)

**Author** Pierre Mabille 

**NYU ID** N10017621 

**Date** March 31, 2016 

### Exercise 1

Define the parameters of the problem.

In [2]:
q1 = [.971 .029 0;
.145 .778 .077;
    0 .508 .492] # 1st transition matrix
q2 = [.871 .029 .1;
.145 .778 .077;
    0 .508 .492] # 2nd transition matrix
d = [1.;
0.;
    0.] # dividend vector (by state)
beta = .98 # discount factor

0.98

Define a function computing the asset price recursively in the case of homogeneous beliefs. The algorithm is a value function iteration, which exploits the contraction property of the $T$ mapping. Then compute (and time the computation of) the price vector (pricing function) for the transition matrix $q_1(x,y)$ and then $q_2(x,y)$.

We find that asset prices are higher in all states when the transition matrix is $q_1(x,y)$, which assigns a higher probability on the first state being persistent. Since this is the state in which the dividend process takes the value 1 (instead of 0 elsewhere), the agent is willing to pay more for the asset because he anticipates to be with a higher probability in the state with higher dividends, since this state is more persistent. Asset prices are higher in all states because the remaining transition probabilities are constant, so that in every state the agent expects higher payoffs from being at some point over the course of his life in the first state with high payoff.

In [3]:
function vfct_iteration(transmat::Array{Float64,2}, div::Array{Float64,1}, beta::Float64; convergencecrit::Float64=1e-12)
    p0 = Array(Float64,length(div)) # initial array of prices (pricing function defined over a finite state space)
    p1 = Array(Float64,length(div)) # image array after applying the mapping T
    supmetric = 1.0 # initialize metric used for convergence
    iter = 1 # count iterations
    while supmetric > convergencecrit
        p1 = beta*transmat*(div + p0) # compute new price vector by applying T to initial price vector
        supmetric = maximum(abs(p1-p0)) # compute sup norm of difference between the two price vectors
        p0 = deepcopy(p1) # update initial price vector (pricing function)
        iter += 1
    end
    return p1, iter
end

vfct_iteration (generic function with 1 method)

In [5]:
@time vfct_iteration(q1,d,beta)

([40.75977457328142,35.9608826474052,34.57200258223242],1359)

 0.170809 seconds (212.51 k allocations: 10.465 MB, 2.76% gc time)


In [6]:
@time vfct_iteration(q2,d,beta)

([23.501134722401407,21.098831099633326,20.283952716361217],1330)

 0.000844 seconds (18.61 k allocations: 1.359 MB)


### Exercise 2

In preamble, let's prove the inequality given as a hint: $$\left| \sup_i a_i - \sup_i b_i \right| \leq \sup_i \left|a_i-b_i \right| \hspace{1cm} \text{i.e.} \hspace{.5cm} \sup_i a_i - \sup_i b_i \leq \sup_i \left|a_i-b_i \right| \hspace{.5cm} \text{and} \hspace{.5cm} \sup_i a_i - \sup_i b_i \geq -\sup_i \left|a_i-b_i \right|  $$
I prove the first inequality, the second has the same proof. Start with any given $j$ in the indexing set. The following series of inequalities are easy to see: $$ a_j - \sup_i b_i \leq a_j - b_j \leq \sup_i (a_i-b_i) \leq \sup_i \left|a_i-b_i \right|  $$
Now, the fact $\forall j$ $a_j - \sup_i b_i \leq \sup_i \left|a_i-b_i \right|$ implies that $\sup_i a_i - \sup_i b_i \leq \sup_i \left|a_i-b_i \right|,$ which is the first inequality.





Let's now go to the bulk of the problem. Assume  $\beta \leq 1$, $d \in \mathbb{R}^S$, $q_i(x,y) \in [0,1]$ and $\sum_{y\in S} q_i(x,y)=1$ for $i=1,2$. Let's prove that $T$ is a contraction, where $T$ is defined, for $x\in S$,  by $$Tp(x)=\beta \max_{i=1,2} \left\lbrace \sum_{y\in S}[d(y)+p(y)]q_i(x,y) \right\rbrace$$

First let's prove that $T:b\mathbb{R}^S \to b\mathbb{R}^S$, i.e. that $||Tp||_{\infty} < \infty$.

As a preliminary result, note that the lemma $|\max_i a_i-\max_i b_i| \leq \max_i |a_i-b_i|$ implies, if we take $b_i=0$ $\forall i$, that $|\max_i a_i| \leq \max_i |a_i|$. I use this result below. I also use the triangle inequality for the absolute value function.

Now take $p\in b\mathbb{R}^S$ and any $x\in S$. 
\begin{eqnarray*}
\left|Tp(x)\right| &=& \beta \left| \max_{i=1,2} \left\lbrace \sum_{y\in S}[d(y)+p(y)]q_i(x,y) \right\rbrace \right| \\
&\leq & \beta   \max_{i=1,2}\left|  \sum_{y\in S}[d(y)+p(y)]q_i(x,y) \right| \\
&\leq & \beta   \max_{i=1,2}  \sum_{y\in S}[\left|d(y)\right|+\left|p(y)\right|]q_i(x,y)  \\
&\leq & \beta   \max_{i=1,2}  \sum_{y\in S}[||d(y)||_{\infty}+||p||_{\infty}]q_i(x,y)  \\
&\leq & \beta  [||d(y)||_{\infty}+||p||_{\infty}]  \hspace{1cm} \forall x\in S \\
\Rightarrow \sup_{x\in S}\left|T p(x) \right| &=& ||Tp||_{\infty} < \infty
\end{eqnarray*}


Second, to complete the (existence and) uniqueness proof, let's prove that $T$ is a uniform contraction. I use the lemma in its original form below. Take any $p,p' \in b\mathbb{R}^S$, and any $x\in S$.
\begin{eqnarray*}
Tp(x)-Tp'(x) &=& \beta \left[\max_{i=1,2} \left\lbrace \sum_{y\in S}[d(y)+p(y)]q_i(x,y) \right\rbrace - \max_{i=1,2} \left\lbrace \sum_{y\in S}[d(y)+p'(y)]q_i(x,y) \right\rbrace \right] \\
\Rightarrow \left| Tp(x)-Tp'(x)\right| &=& \beta \left|\max_{i=1,2} \left\lbrace \sum_{y\in S}[d(y)+p(y)]q_i(x,y) \right\rbrace - \max_{i=1,2} \left\lbrace \sum_{y\in S}[d(y)+p'(y)]q_i(x,y) \right\rbrace \right| \\
&\leq & \beta\max_{i=1,2}\left|  \sum_{y\in S}[d(y)+p(y)-d(y)-p'(y)]q_i(x,y) \right| \\
&= & \beta\max_{i=1,2}\left|  \sum_{y\in S}[p(y)-p'(y)]q_i(x,y) \right| \\
&\leq & \beta\max_{i=1,2}  \sum_{y\in S}\left|p(y)-p'(y)\right|q_i(x,y)  \\
&\leq & \beta\max_{i=1,2}  \sum_{y\in S}\left|\left|p-p'\right|\right|_{\infty}q_i(x,y)  \\
&= & \beta\left|\left|p-p'\right|\right|_{\infty} \hspace{1cm} \forall x\in S  \\
\Rightarrow \left|\left|Tp-Tp' \right|\right|_{\infty} &\leq& \beta\left|\left|p-p'\right|\right|_{\infty}. \hspace{.5cm} QED.
\end{eqnarray*}

### Exercise 3

Define a function computing the asset price recursively in the case of heterogeneous beliefs. The algorithm is the same as with homogeneous beliefs, except that the function takes 2 transition matrices as arguments, and the $T$ mapping changes.

We compute the pricing function for the transition matrices defined in Exercise 1. We find that prices are the same as in the case where the dividend process follows the transition matrix $q_1(x,y)$. This means that the more optimistic agent (who expects the state with the high payoff to be more persistent) prices the asset. 

In [7]:
function vfct_iteration_hb(transmat1::Array{Float64,2}, transmat2::Array{Float64,2}, div::Array{Float64,1}, beta::Float64; convergencecrit::Float64=1e-12)
    p0 = Array(Float64,length(div))
    p1 = Array(Float64,length(div))
    supmetric = 1.0
    iter = 1
    while supmetric > convergencecrit
        for i = 1:length(div) # instead of looping could define function taking the max by columns 
            p1[i] = beta*(max(transmat1[i,:]*(div + p0), transmat2[i,:]*(div + p0)))[1] # line by line maximum
        end
        supmetric = maximum(abs(p1-p0))
        p0 = deepcopy(p1)
        iter += 1
    end
    return p1, iter
end

vfct_iteration_hb (generic function with 1 method)

In [8]:
@time vfct_iteration_hb(q1,q2,d,beta)

([40.75977457328136,35.96088264740514,34.57200258223237],1359)

 0.220295 seconds (165.53 k allocations: 8.136 MB, 2.53% gc time)
